In [58]:
# Import necessary libraries
import sqlite3
import random
from datetime import datetime, timedelta
from faker import Faker
import pandas as pd
import uuid

In [59]:
# Initialize the Faker library for data generation
fake = Faker()

# Establish a connection to the SQLite database
# This step creates the database file if it doesn't already exist
conn = sqlite3.connect('databases/random_data.db')
cursor = conn.cursor()

### 🧑🏻Creating User Table

This section creates a table named `users` in the SQLite database to store user information. It includes fields like `user_id`, `name`, `dob` (date of birth), `country`, `income`, `occupation`, and `user_activation_date`. If the table already exists, it is dropped and recreated to start fresh.


In [60]:
# Drop table if it already exists
cursor.execute('DROP TABLE IF EXISTS users')

# Create the users table
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    user_id TEXT PRIMARY KEY,
    name TEXT,
    dob DATE,
    country TEXT,
    income INTEGER,
    occupation TEXT,
    user_activation_date DATE
)
''')

print("Table successfully created.")

Table successfully created.


### Populate the Users Table with Fake Data

This section populates the `users` table with fake data for demonstration purposes. It generates random data for each field using the Faker library and a UUID for each `user_id`.


In [61]:
# Populate the users table with random data, including a UUID for each user_id
for _ in range(10000):
    user_id = str(uuid.uuid4())  # Generate a UUID for the user_id
    name = fake.name()
    dob = fake.date_of_birth(minimum_age=18, maximum_age=99).isoformat()
    country = fake.country()
    # Generate a datetime object including time, then format it to the desired string format
    user_activation_date = fake.date_time_between(start_date='-3y', end_date='now').strftime('%Y-%m-%d %H:%M:%S')
    income = random.randint(5000, 100000)  # Assuming income range
    occupation = fake.job()

    cursor.execute('''
    INSERT INTO users (user_id, name, dob, country, user_activation_date, income, occupation)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (user_id, name, dob, country, user_activation_date, income, occupation))

conn.commit()

# Close the connection
conn.close()

print("Table setup and data insertion complete.")

Table setup and data insertion complete.


### 💵 Creating the Transactions Table

This section outlines the steps to create a `transactions` table in the database. This table will include a unique `transaction_id`, a `user_id` to link to the `users` table, a `transaction_amount`, and a `transaction_date`, capturing the details of each transaction made by users.


In [62]:
# Connect to SQLite database
conn = sqlite3.connect('databases/random_data.db')
cursor = conn.cursor()

# Drop table if it already exists
cursor.execute('DROP TABLE IF EXISTS transactions')

# Create Transactions table
cursor.execute('''
CREATE TABLE IF NOT EXISTS transactions (
    trx_id INTEGER PRIMARY KEY AUTOINCREMENT,
    transaction_type TEXT,
    amount INTEGER,
    user_id TEXT,
    recipient_name TEXT,
    transaction_create_date DATE,
    FOREIGN KEY(user_id) REFERENCES users(user_id)
)
''')

print("Table successfully created.")

Table successfully created.


### Populating the Transactions Table with Fake Data

To simulate user transactions, this part of the script inserts fake transaction data into the `transactions` table. Each transaction is linked to a user by `user_id`, and includes a randomly generated `transaction_amount` and `transaction_date`.


In [63]:
# Function to generate a random datetime between `start` and `end`
def random_date(start, end):
    """Generate a random datetime between `start` and `end`."""
    time_between_dates = end - start
    seconds_between_dates = time_between_dates.total_seconds()
    random_seconds = random.randint(0, int(seconds_between_dates))
    return start + timedelta(seconds=random_seconds)

start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 2, 10)

# Connect to the SQLite database
conn = sqlite3.connect('databases/random_data.db')
cursor = conn.cursor()

# Fetch UUIDs from the users table
cursor.execute("SELECT user_id FROM users")
user_ids = [row[0] for row in cursor.fetchall()]

# Define additional recipient options
additional_recipients = [
    "Netflix", "Spotify", "Amazon Prime", "Disney+", "Apple Music",
    "YouTube Premium", "HBO Max", "Showmax", "Uber", "Airbnb",
    "Starbucks", "McDonald's", "Adidas", "Nike", "Microsoft",
    "Google", "Facebook", "Tesla", "Samsung", "Sony"
]

# Generate and insert transactions data for each user
for user_id in user_ids:
    num_transactions = random.randint(0, 100)  # Number of transactions for this user
    
    for _ in range(num_transactions):
        transaction_type = random.choice(['Deposit', 'Withdrawal', 'Transfer'])
        amount = random.randint(100, 100000)  # Random amount in ZAR
        recipient_name = random.choice([fake.name()] + additional_recipients)  # Enhanced recipient selection
        transaction_create_date = random_date(start_date, end_date).strftime('%Y-%m-%d %H:%M:%S')  # Adjusted format
        
        # Insert transaction data
        cursor.execute('''
        INSERT INTO transactions (transaction_type, amount, user_id, recipient_name, transaction_create_date)
        VALUES (?, ?, ?, ?, ?)
        ''', (transaction_type, amount, user_id, recipient_name, transaction_create_date))

conn.commit()
conn.close()

print("Table setup and data insertion complete.")

Table setup and data insertion complete.


### 🗂️ Creating the Subscriptions Table

This section is responsible for creating a `subscriptions` table in the database. This table stores subscription details, including a unique `subscription_id` and the `subscription_name`. It represents the various subscriptions a user can have.


In [64]:
# Connect to the SQLite database
conn = sqlite3.connect('databases/random_data.db')
cursor = conn.cursor()

# Drop table if it already exists
cursor.execute('DROP TABLE IF EXISTS subscription_services')

# Create the subscription services table
cursor.execute('''
CREATE TABLE IF NOT EXISTS subscription_services (
    subscription_name TEXT,
    subscription_id INTEGER PRIMARY KEY AUTOINCREMENT,
    subscription_price REAL,
    subscription_tier TEXT
)
''')

conn.commit()
conn.close()

print("Table successfully created.")

Table successfully created.


### Populating the Subscription Services Table
In this section, we connect to an SQLite database and insert data into a `subscription_services` table. This table is designed to store information about various subscription services, including their names, prices, and tiers. The goal is to simulate a realistic set of subscription options that users might have, reflecting the diversity in services available in the market.

In [65]:
# Connect to the SQLite database
conn = sqlite3.connect('databases/random_data.db')
cursor = conn.cursor()

# Define a list of example subscription services
subscription_services = [
    {"name": "Netflix", "price": random.uniform(99, 199), "tier": random.choice(["Basic", "Standard", "Premium"])},
    {"name": "Showmax", "price": random.uniform(49, 149), "tier": random.choice(["Basic", "Standard", "Premium"])},
    {"name": "Amazon Prime", "price": random.uniform(79, 159), "tier": random.choice(["Basic", "Standard", "Premium"])},
    {"name": "Spotify", "price": random.uniform(59, 119), "tier": random.choice(["Free", "Premium"])},
    {"name": "Disney+", "price": random.uniform(69, 129), "tier": random.choice(["Basic", "Standard", "Premium"])},
    {"name": "HBO Max", "price": random.uniform(99, 199), "tier": random.choice(["Basic", "Standard", "Premium"])},
    {"name": "Apple Music", "price": random.uniform(59, 119), "tier": random.choice(["Individual", "Family", "Student"])},
    {"name": "YouTube Premium", "price": random.uniform(59, 119), "tier": random.choice(["Basic", "Premium"])}
]

# Insert data into the subscription services table
for service in subscription_services:
    cursor.execute('''
    INSERT INTO subscription_services (subscription_name, subscription_price, subscription_tier)
    VALUES (?, ?, ?)
    ''', (service["name"], round(service["price"], 2), service["tier"]))

conn.commit()
conn.close()

print("Table setup and data insertion complete.")

Table setup and data insertion complete.


### 🔗 Creating the User Subscriptions Junction Table

Here, we create a `user_subscriptions` table that acts as a junction (or linking) table between `users` and `subscriptions`. This design allows for a many-to-many relationship, where users can have multiple subscriptions, and each subscription can be associated with multiple users.


In [66]:
# Connect to the SQLite database
conn = sqlite3.connect('databases/random_data.db')
cursor = conn.cursor()

# Drop table if it already exists
cursor.execute('DROP TABLE IF EXISTS user_subscriptions')

# Create the user_subscriptions table
cursor.execute('''
CREATE TABLE IF NOT EXISTS user_subscriptions (
    user_id TEXT,
    subscription_id INTEGER,
    PRIMARY KEY (user_id, subscription_id),
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (subscription_id) REFERENCES subscriptions_metadata(subscription_id)
)
''')

print("Table successfully created.")

Table successfully created.


### Populating the Subscriptions Table with Data

This segment inserts predefined subscription data into the `subscriptions` table. Each subscription has a unique identifier and a name.


In [67]:
# Fetch all user_ids from the users table
cursor.execute("SELECT user_id FROM users")
user_ids = [row[0] for row in cursor.fetchall()]

# Fetch all subscription_ids from the subscriptions_metadata table
cursor.execute("SELECT subscription_id FROM subscription_services")
subscription_ids = [row[0] for row in cursor.fetchall()]

# Reconnect to the database
conn = sqlite3.connect('databases/random_data.db')
cursor = conn.cursor()

# Randomly assign subscriptions to users
for user_id in user_ids:
    # Each user can have between 1 and 5 subscriptions for demonstration
    chosen_subscriptions = random.sample(subscription_ids, random.randint(1, 5))
    for subscription_id in chosen_subscriptions:
        cursor.execute('''
        INSERT INTO user_subscriptions (user_id, subscription_id)
        VALUES (?, ?)
        ''', (user_id, subscription_id))

print("Table setup and data insertion complete.")

conn.commit()
conn.close()

Table setup and data insertion complete.


### 👀 Database Checks
Checking that all the tables are correctly created and data insertions were successful.

In [68]:
# Connect to the SQLite database
conn = sqlite3.connect('databases/random_data.db')
cursor = conn.cursor()

# Querys
user_query = '''
SELECT * 
FROM users'''

trx_query = '''
SELECT * 
FROM transactions;'''

sub_service_query = '''
SELECT * 
FROM subscription_services'''

user_subs_query = '''
SELECT * 
FROM user_subscriptions'''

# Read the data into a pandas DataFrames
user_df = pd.read_sql_query(user_query, conn)
transactions_df = pd.read_sql_query(trx_query, conn)
subscription_service_df = pd.read_sql_query(sub_service_query, conn)
user_subscriptions_df = pd.read_sql_query(user_subs_query, conn)

# Close the connection
conn.close()

In [69]:
user_df.head(10)

,user_id,name,dob,country,income,occupation,user_activation_date
0,f231a6e6-9e8d-4287-8ee6-c0ca8321e460,Karen Bishop,1946-10-25,Iraq,23044,Toxicologist,2023-12-06 02:45:04
1,23b72926-f4ec-4abd-9288-fd1b66f1e6ff,Tonya Brooks,1992-09-09,Libyan Arab Jamahiriya,54405,Magazine features editor,2022-08-11 02:26:10
2,881b6144-9bdc-4e76-b8bb-edc520265553,Louis Smith Jr.,1945-12-04,Venezuela,76958,Metallurgist,2024-01-05 13:01:21
3,e2c7a43f-747f-401d-bfb9-fac5788cb4b9,Warren Rodgers,2004-12-31,North Macedonia,79161,Broadcast engineer,2021-09-19 02:25:29
4,a7fc1cbc-9924-473e-9ff9-3a959d4737d0,Teresa Christian,1929-06-23,Bouvet Island (Bouvetoya),37378,Acupuncturist,2022-05-04 17:55:42
5,631bd714-3f6f-4e7e-bcac-5142530d880a,Jeremy Figueroa,1976-01-27,Belgium,26153,"Scientist, research (life sciences)",2021-10-21 07:37:10
6,7f4af4f5-f9a0-4cb5-952f-911bbc252c71,Mr. Kenneth Erickson MD,1976-01-21,Reunion,95402,Operational investment banker,2022-07-23 12:49:15
7,47a3c484-fa9b-4545-99e1-e4d162262b32,Harold Clark,1978-04-23,Kenya,83397,Special educational needs teacher,2021-06-07 13:27:35
8,0c619e16-d897-4d2c-9323-58ca506196b9,Paul Stafford,2001-10-10,Micronesia,23151,Airline pilot,2023-01-05 21:31:33
9,5a257629-1f02-4a0e-a8f8-ba926251c24b,Cory Harrell,1943-04-02,Liechtenstein,38685,Museum education officer,2022-11-22 06:56:46


In [71]:
transactions_df.head(10)

,trx_id,transaction_type,amount,user_id,recipient_name,transaction_create_date
369894,369895,Deposit,11868,be3aaad3-3cdd-4cf7-9245-3c6bf998e800,Facebook,2020-01-01 00:07:30
320403,320404,Transfer,87407,a49d1b24-d8a5-4b6e-9fcc-d8424794af7d,Netflix,2020-01-01 00:14:34
375456,375457,Transfer,68128,c0cdf1f2-0f39-4cfc-82c6-a7a39fb11b53,Microsoft,2020-01-01 00:15:11
186897,186898,Transfer,13963,604688dc-ae3a-4246-8530-89825624b0c4,Showmax,2020-01-01 00:15:26
399370,399371,Transfer,229,cc897bce-7a13-4aaf-a9a9-650ca1e90abd,Google,2020-01-01 00:15:53
80697,80698,Withdrawal,33474,298f245c-6ca0-4f80-b348-6de498ab1cf6,HBO Max,2020-01-01 00:17:49
477568,477569,Transfer,40405,f3ffc435-2658-473e-afbf-91251ceae6a7,Jennifer Baldwin,2020-01-01 00:18:03
142225,142226,Deposit,3185,4a07ae91-4fb9-4aa7-b9d0-587be4febb2b,Google,2020-01-01 00:19:26
198274,198275,Deposit,93575,65eb2a80-d727-4460-a6e0-1adff2eca903,Facebook,2020-01-01 00:22:01
28777,28778,Deposit,38585,0e38bd34-50d4-4c58-b9d6-1c05316ae251,Facebook,2020-01-01 00:29:14


In [72]:
subscription_service_df.head(10)

,subscription_name,subscription_id,subscription_price,subscription_tier
0,Netflix,1,131.44,Basic
1,Showmax,2,147.37,Standard
2,Amazon Prime,3,131.19,Standard
3,Spotify,4,92.27,Premium
4,Disney+,5,121.15,Standard
5,HBO Max,6,104.80,Premium
6,Apple Music,7,114.16,Individual
7,YouTube Premium,8,104.14,Basic


In [73]:
user_subscriptions_df.head(10)

,user_id,subscription_id
0,00005bee-9c55-43d2-be3e-29000026e85f,8
1,00005bee-9c55-43d2-be3e-29000026e85f,4
2,00005bee-9c55-43d2-be3e-29000026e85f,2
3,00005bee-9c55-43d2-be3e-29000026e85f,1
4,000fac06-78dd-4187-9458-08d1ddb0f95a,1
5,000fac06-78dd-4187-9458-08d1ddb0f95a,2
6,000fac06-78dd-4187-9458-08d1ddb0f95a,7
7,000fac06-78dd-4187-9458-08d1ddb0f95a,6
8,000fac06-78dd-4187-9458-08d1ddb0f95a,4
9,00145384-eb4a-4da0-b067-67c7b6386342,1
